In [1]:
# MNE-Python recommends using qt.
%matplotlib qt

# Analysis workflows

Look into the [PREP pipeline](http://dx.doi.org/10.3389/fninf.2015.00016) as an example.  
What [MNE recommends](http://martinos.org/mne/stable/auto_tutorials/plot_artifacts_correction_ica.html#what-if-we-don-t-have-an-eog-channel) if you do not have an EOG channel.

It is a good idea to visually check the data throughout analysis.

QUESTIONS
---------
- Should we filter all of the data, remove bad segments, and then ICA all of the data? Or could we filter --> remove bad segments/channels --> ICA per subject? ICA in MNE is fast enough (~1 min) that I think it is better to do everything per subject.

Time-Frequency
--------------

__By subject__

1. Load data
1. Bandpass filter (one at a time?)
1. Reject artifacts (blinks, flat channels, excessively noisy channels)
    - ICA
    - To detect noisy/blink components with ICA in many subjects, try `mne.preprocessing.corrmap()`
    - how to detect flat channels?
1. Epoch
    - reject based on peak-to-peak amplitude using `reject` parameter.

__By group__

1. Statistical analyses

Automated artifact rejection
----------------------------

- SSP (projection vectors)
- use `mne.preprocessing.find_eog_events()`
- use `mne.preprocessing.run_ica()`
- use `mne.preprocessing.ICA.detect_artifacts()`
- use `mne.preprocessing.corrmap()`
- use flat signal detection
- specify `reject` parameter of mne.Epochs

We can compute and save ICA solution and save if we want to compute
the ICA (longest step) all at once, let's say overnight.
Make a script to compute ICA for everyone. We would have to load 
the raw file and the corresponding ICA solution when removing noisy components.

Use `mne.io.Raw.set_montage()` to add the electrode location information.

In [2]:
import os
import glob
import re

# import numpy as np

from mne import (
    set_config, set_log_level, io, compute_proj_raw,
    pick_types, make_fixed_length_events, Epochs, )
from mne.utils import ProgressBar
from mne.preprocessing import ICA

In [12]:
# Change working directory to data folder.
# path = os.path.join('data')
os.chdir('data')

OSError: [Errno 2] No such file or directory: 'data'

In [24]:
os.path.realpath(__file__)

NameError: name '__file__' is not defined

In [ ]:
# # Configure some MNE settings.
# # List of keys can be found in mne.utils.known_config_types
# set_config('MNE_STIM_CHANNEL', 'STI101', set_env=True)

# Change how much MNE-Python talks.
# Verbose can be "DEBUG", "INFO", "WARNING", "ERROR", or "CRITICAL".
set_log_level(verbose='warning')

In [ ]:
# How to deal with multiple files per subject/task?
raw_fnames = [f for f in glob.glob("1_raw/*sudoku*.set")]
print "Found", len(raw_fnames), "matching files"

# In the future, write something to clean up the filenames
# and perhaps to include some extra meta-data in the files.

# Workflow for one subject

## Load raw data

In [ ]:
# Load raw data.
raw = io.read_raw_eeglab(raw_fnames[1], preload=True)
# If prelaod=False, you can call `raw.load_data()` when necessary.
# If you need to remove the instance from memory, just delete the variable
# with raw = None

# Set filename, which will make it easier to save with informative filenames.
raw.info['filename'] = 'NEU_001-sudoku'

# Identify which data you want to work with.
# Some files have multiple types of data, so pick_types() becomes very useful.
raw.pick_types(meg=False, eeg=True, stim=False)

# Crop out the first 20 seconds of data.
raw.crop(tmin=20.0)

## Bandpass filter

In [ ]:
l_freq = 0.5
h_freq = 40.

# The output is different between simultaneous bandpass filtering
# and separate filtering.

# # High- and low-pass filter simultaneously.
# raw.filter(l_freq=l_freq, h_freq=h_freq, picks=picks, phase='zero', fir_window='hamming', 
#            l_trans_bandwidth='auto', h_trans_bandwidth='auto', 
#            filter_length='auto')

# High- and low-pass filter separately.
raw.filter(l_freq=l_freq, h_freq=None, phase='zero', fir_window='hamming', 
           l_trans_bandwidth='auto', h_trans_bandwidth='auto', 
           filter_length='auto')
raw.filter(l_freq=None, h_freq=h_freq, phase='zero', fir_window='hamming', 
           l_trans_bandwidth='auto', h_trans_bandwidth='auto', 
           filter_length='auto')

# Indicate in the filename that we filtered
raw.info['filename'] += '-firfilt'
# raw.save(raw.info['filename'] + '.fif')

In [ ]:
# Visualize data to remove bad channels before the next steps
raw.plot()

In [ ]:
print "Removed channel(s)", raw.info['bads']
raw.pick_types(meg=False, eeg=True)

## SSP vs using average reference

To me, it looks better to use the average reference.

In [ ]:
projection = compute_proj_raw(raw, n_grad=0, n_mag=0, n_eeg=1)
raw.copy().add_proj(projection).plot(title="Using spatial space projection vector")

In [ ]:
raw.copy().set_eeg_reference(None).plot(title="Using average reference")

In [ ]:
# Use an average reference. None makes it use an average.
# Average reference not recommended for anything below 64 channels.
raw.set_eeg_reference(None)

# Ideally, we would not remove channels after setting the average reference.

## Remove bad segments

In [ ]:
# Inspect data.
raw.plot(n_channels=len(raw.ch_names))

In [ ]:
# How do you manually add annotations?

# Remove flat channels. You can remove these by selecting them on the plot,
# but is there automated detection of flat channels?

raw.info['bads'] += []

print "Channels removed:", raw.info['bads']
# Removes channels in raw.info['bads'] by default.
raw.pick_types(meg=False, eeg=True)

In [ ]:
# Inspect data to see the effect of any changes made.
raw.plot(n_channels=len(raw.ch_names))

In [ ]:
# # The following attempts to repair bad channels.
# # https://www.martinos.org/mne/stable/manual/channel_interpolation.html
# raw.interpolate_bads(reset_bads=False)

In [ ]:
# Modify filename to indicate that we removed bad segments.
raw.info['filename'] += '-reref-cleaned'

## Independent component analysis (ICA)

In [ ]:
# Use extended-infomax algorithm. EEGLAB also uses this algorithm.
ica = ICA(method='extended-infomax').fit(raw)

# # Save ICA solution.
# ica.save(raw.info['filename'] + '-ica.fif')

In [ ]:
# Click on a component to plot detailed information about it.
ica.plot_components(inst=raw)  # Plot topography of ICA components.
ica.plot_sources(raw, start=0, stop=10)  # Plot timecourse of components.
raw.plot(duration=10)  # Plot raw data to compare to ICA sources.

In [ ]:
raw.pick_types(meg=False, eeg=True)

# Ideally, you would manually check for EOG components.
# But we can compare the EOG components that a rater finds versus
# the components this function finds.

# Check Fp1 and Fp2, but we must do this separately because find_bads_eog()
# does not accept list type for the ch_name parameter.
# We set verbose here to only print errors, because otherwise it will
# print warnings about changes in default values for the upcoming version.
eog_components, eog_scores = ica.find_bads_eog(raw, ch_name='Fp2', verbose='error')

# Print the components that it thinks are blinks, and print the 
# correlation scores of those components.
for c in eog_components:
    print c, '\t', eog_scores[c]
if not eog_components:
    print "Could not find any EOG components."

In [ ]:
# You can specify ica.exclude manually, or you can mark
# the components you want to remove in the plot of ICA sources.
# Components in ica.exclude will be removed with the apply() method.

# [5, 21, 22]

# Manually:
# ica.exclude += [6, 7, 10]
print "Components marked for removal:", ica.exclude

In [ ]:
# apply() zeros out all components in ica.exclude,
# and it operates in-place on the instance of Raw or Epochs.
# Use a copy of the Raw or Epochs instance if you do not want
# apply() to operate in-place.
raw_cleaned_with_ica = ica.apply(raw.copy())

In [ ]:
raw_cleaned_with_ica.plot()

In [ ]:
# Remove any bad channels selected while viewing the plot.
print "Channels removed:", raw_cleaned_with_ica.info['bads']
raw_cleaned_with_ica.pick_types(meg=False, eeg=True)

raw_cleaned_with_ica.info['filename'] += '-pruned_with_ica'
# raw_cleaned_with_ica.save(raw.info['filename'])

## Epoch

In [ ]:
# The trigger name we want to give to the epochs.
event_id = 1

# The desired length of each epoch.
epoch_duration = 30.

events = make_fixed_length_events(
    raw_cleaned_with_ica, event_id, start=0, stop=None,
    duration=epoch_duration)

# # If our data had included events, we would have used the following.
# events = mne.find_events(raw)


# What makes the epochs look so wrong? Is it the EEG reference? Detrending? DC offset?
# Removing the DC offset seems to help.

# Create epochs based on the events created above.
epochs = Epochs(
    raw_cleaned_with_ica, events, event_id=event_id, tmin=0.0, tmax=epoch_duration, 
    baseline=None, preload=True, detrend=0, add_eeg_ref=False)
epochs

In [ ]:
# epochs.plot()

# Plot one epoch.
epochs[0].plot()

In [ ]:
epochs.info['filename'] += '-epochs'
epochs.save(epochs.info['filename'])

# Power

Look for theta desync from anything behind Cz (e.g., PO1/2, Oz, O1/2)
Check for differences between rest and /_/_ (attention tasks).
- Do this for each channel, and check whether multiple channels are significantly different.

Multiple comparisons to account for multiple channels / time.

Look into `mne.stats.spatio_temporal_permutation_cluster_1samp_test()`
Use `stats.fdr_correction()` with p-value of 0.05.